In [1]:
import pandas as pd
import numpy as np

### Giải nghĩa thuật ngữ

#### ZHVI - Zillow Home Value Index
- **giá trị nhà** & thay đổi của thị trường
- measure typical home value & market changes across states & housing type
- 2 housing types: condo / co-ops vs. single-family residences (SFR)

#### Inventory
- **for-sale inventory**: đếm số lượng unique listings mà active trong cả tháng đó -> 1 căn có thể được tính nhiều lần trong nhiều tháng liên tiếp
- **new listings**: có bn listings mới lên thị trường trong tháng đó -> 1 căn sẽ chỉ được tính 1 lần
- **newly pending listings**: số listings chuyển từ trạng thái "đang rao bán" thành trạng thái "pending" trong tháng đó (pending maybe là đã được cọc)
- **days to pending**: thời gian 1 căn chuyển từ trạng thái "đang rao bán" (thời gian đầu có trạng thái này) sang "pending" -> 1 căn có thể chuyển đi chuyển lại giữa 2 trạng thái này, nên lấy mốc thời gian đầu ở trạng thái "đang rao bán" & k tính giai đoạn có status là "in-contract" (k rõ lắm)


#### List & Sale prices
- **median list price**: giá đăng trên các trang bđs / giá tới công chúng
- **median sale price**: giá thực bán
- **sale-to-list ratio**: tỉ lệ giữa giá thực bán / giá đăng bán (final)
- **percent of sales under / over list**: % các căn đc bán có giá thực bán nhỏ hơn / lớn hơn giá đăng bán (final), k tính những căn có giá thực bán = giá đăng bán
- **days to close**: số ngày từ lúc listing đó từ "pending" -> sale date (ngày thực bán?)

### ZHVI - Condo

In [2]:
# ZHVI - Zillow Home Value Index: home value + market change of condo / co-op
path = 'zhvi-condo-state.csv'
df_condo = pd.read_csv(path)
df_condo.drop('StateName', axis=1, inplace=True)
df_condo = df_condo.melt(df_condo.columns[0:4], var_name='Month', value_name='HVI')
df_condo

,RegionID,SizeRank,RegionName,RegionType,Month,HVI
0,9,0,California,state,2000-01-31,181242.0
1,54,1,Texas,state,2000-01-31,105596.0
2,14,2,Florida,state,2000-01-31,95534.0
3,43,3,New York,state,2000-01-31,219160.0
4,47,4,Pennsylvania,state,2000-01-31,98567.0
...,...,...,...,...,...,...
13867,37,46,North Dakota,state,2022-08-31,184304.0
13868,3,47,Alaska,state,2022-08-31,267372.0
13869,12,48,District of Columbia,state,2022-08-31,463427.0
13870,58,49,Vermont,state,2022-08-31,327308.0


In [3]:
lst = ['condo'] * len(df_condo)
df_condo.insert(3,'ListingType',lst,True)
df_condo

,RegionID,SizeRank,RegionName,ListingType,RegionType,Month,HVI
0,9,0,California,condo,state,2000-01-31,181242.0
1,54,1,Texas,condo,state,2000-01-31,105596.0
2,14,2,Florida,condo,state,2000-01-31,95534.0
3,43,3,New York,condo,state,2000-01-31,219160.0
4,47,4,Pennsylvania,condo,state,2000-01-31,98567.0
...,...,...,...,...,...,...,...
13867,37,46,North Dakota,condo,state,2022-08-31,184304.0
13868,3,47,Alaska,condo,state,2022-08-31,267372.0
13869,12,48,District of Columbia,condo,state,2022-08-31,463427.0
13870,58,49,Vermont,condo,state,2022-08-31,327308.0


### ZHVI - SFR

In [4]:
path_2 = 'zhvi-sfr-state.csv'
df_sfr = pd.read_csv(path_2)
df_sfr.drop('StateName', axis=1, inplace=True)
df_sfr.head()

,RegionID,SizeRank,RegionName,RegionType,2000-01-31,2000-02-29,2000-03-31,2000-04-30,2000-05-31,2000-06-30,...,2021-11-30,2021-12-31,2022-01-31,2022-02-28,2022-03-31,2022-04-30,2022-05-31,2022-06-30,2022-07-31,2022-08-31
0,9,0,California,state,201553.0,202407.0,203471.0,205707.0,208093.0,210570.0,...,733082.0,739882.0,750166.0,762417.0,777520.0,792039.0,802957.0,805268.0,802064.0,792787.0
1,54,1,Texas,state,114514.0,114633.0,114670.0,114874.0,114920.0,114985.0,...,276591.0,280894.0,285598.0,291081.0,296533.0,302617.0,308113.0,312844.0,315337.0,316247.0
2,14,2,Florida,state,119943.0,120313.0,120697.0,121429.0,122119.0,122792.0,...,349739.0,356885.0,364805.0,373079.0,383122.0,394118.0,405094.0,414295.0,420497.0,423929.0
3,43,3,New York,state,139891.0,140582.0,141157.0,142381.0,143476.0,144599.0,...,346636.0,349509.0,352754.0,356451.0,361006.0,365681.0,370538.0,374511.0,377539.0,379307.0
4,47,4,Pennsylvania,state,106781.0,107140.0,107434.0,108001.0,108487.0,108926.0,...,249151.0,251426.0,254019.0,256850.0,259528.0,262182.0,264617.0,266782.0,268296.0,269306.0


In [5]:
df_sfr = df_sfr.melt(df_sfr.columns[0:4], var_name='Month', value_name='HVI')
lst = ['sfr'] * len(df_sfr)
df_sfr.insert(3, 'ListingType', lst, True)
df_sfr


,RegionID,SizeRank,RegionName,ListingType,RegionType,Month,HVI
0,9,0,California,sfr,state,2000-01-31,201553.0
1,54,1,Texas,sfr,state,2000-01-31,114514.0
2,14,2,Florida,sfr,state,2000-01-31,119943.0
3,43,3,New York,sfr,state,2000-01-31,139891.0
4,47,4,Pennsylvania,sfr,state,2000-01-31,106781.0
...,...,...,...,...,...,...,...
13867,37,46,North Dakota,sfr,state,2022-08-31,284545.0
13868,3,47,Alaska,sfr,state,2022-08-31,345741.0
13869,12,48,District of Columbia,sfr,state,2022-08-31,811792.0
13870,58,49,Vermont,sfr,state,2022-08-31,367290.0


### Combine ZHVI data (by Listing Type)

In [6]:
df = pd.concat([df_condo, df_sfr])
df

,RegionID,SizeRank,RegionName,ListingType,RegionType,Month,HVI
0,9,0,California,condo,state,2000-01-31,181242.0
1,54,1,Texas,condo,state,2000-01-31,105596.0
2,14,2,Florida,condo,state,2000-01-31,95534.0
3,43,3,New York,condo,state,2000-01-31,219160.0
4,47,4,Pennsylvania,condo,state,2000-01-31,98567.0
...,...,...,...,...,...,...,...
13867,37,46,North Dakota,sfr,state,2022-08-31,284545.0
13868,3,47,Alaska,sfr,state,2022-08-31,345741.0
13869,12,48,District of Columbia,sfr,state,2022-08-31,811792.0
13870,58,49,Vermont,sfr,state,2022-08-31,367290.0


In [7]:
df.to_excel("f_zhvi.xlsx", sheet_name='zhvi_union', encoding='utf-8', index=False)